# 音声合成プログラム

このノートブックでは、`sbv2_bindings` パッケージを使用して音声合成を行います。必要なモデルをダウンロードし、ユーザーが入力したテキストから音声を生成します。音声合成が終わったら、再度テキストの入力を求め、ユーザーが終了するまで繰り返します。

In [ ]:
# 必要なパッケージのインストール
%pip install sbv2_bindings

# 必要なモジュールのインポート
import os
import urllib.request
import time
from sbv2_bindings import TTSModel

## モデルのダウンロード

モデルファイルとトークナイザーをダウンロードします。ユーザーが独自のモデルを使用したい場合は、該当するURLまたはローカルパスを指定してください。

In [ ]:
# モデルの URL またはローカルパスの指定
user_sbv2_model_url = ""  # カスタムモデルのURLがあればここに指定
user_sbv2_model_path = ""  # カスタムモデルのローカルパスがあればここに指定

# モデル用のディレクトリを作成
model_dir = 'models'
os.makedirs(model_dir, exist_ok=True)

# ダウンロードするファイルの URL
file_urls = [
    "https://huggingface.co/googlefan/sbv2_onnx_models/resolve/main/tokenizer.json",
    "https://huggingface.co/googlefan/sbv2_onnx_models/resolve/main/deberta.onnx",
]

# モデルのパス決定
if user_sbv2_model_path:
    sbv2_model_path = user_sbv2_model_path  # ローカルモデルのパスを使用
elif user_sbv2_model_url:
    sbv2_model_filename = os.path.basename(user_sbv2_model_url)
    sbv2_model_path = os.path.join(model_dir, sbv2_model_filename)
    file_urls.append(user_sbv2_model_url)
else:
    # デフォルトのモデルを使用
    sbv2_model_filename = "tsukuyomi.sbv2"
    sbv2_model_path = os.path.join(model_dir, sbv2_model_filename)
    file_urls.append("https://huggingface.co/googlefan/sbv2_onnx_models/resolve/main/tsukuyomi.sbv2")

# ファイルをダウンロード
for url in file_urls:
    file_name = os.path.join(model_dir, os.path.basename(url))
    if not os.path.exists(file_name):
        print(f"{file_name} をダウンロードしています...")
        urllib.request.urlretrieve(url, file_name)
    else:
        print(f"{file_name} は既に存在します。")

# ダウンロードまたは使用するファイルを確認
print("\n使用するファイル:")
for file in os.listdir(model_dir):
    print(file)

## モデルの読み込みと音声合成

モデルを読み込み、ユーザーが入力したテキストから音声を生成します。話者名は使用する `.sbv2` ファイル名から自動的に取得します。音声合成が終わったら、再度テキストの入力を求め、ユーザーが終了するまで繰り返します。

In [ ]:
# 音声合成の実行
def main():
    try:
        print("\nモデルを読み込んでいます...")
        model = TTSModel.from_path(
            os.path.join(model_dir, "deberta.onnx"),
            os.path.join(model_dir, "tokenizer.json")
        )
        print("モデルの読み込みが完了しました！")
    except Exception as e:
        print(f"モデルの読み込みに失敗しました: {e}")
        return

    # 話者名を取得（.sbv2 ファイル名の拡張子を除いた部分）
    speaker_name = os.path.splitext(os.path.basename(sbv2_model_path))[0]
    
    # 指定されたモデルのパスを使用
    try:
        model.load_sbv2file_from_path(speaker_name, sbv2_model_path)
        print(f"話者 '{speaker_name}' のセットアップが完了しました！")
    except Exception as e:
        print(f"SBV2ファイルの読み込みに失敗しました: {e}")
        return

    # 音声合成を繰り返し実行
    while True:
        # 合成したいテキストをユーザーから入力
        user_input = input("\n音声合成したいテキストを入力してください（終了するには 'exit' と入力）: ")
        
        if user_input.strip().lower() == 'exit':
            print("音声合成を終了します。")
            break

        # 出力ファイル名
        output_file = "output.wav"

        # 音声合成を実行
        try:
            print("\n音声合成を開始します...")
            start_time = time.time()

            audio_data = model.synthesize(user_input, speaker_name, 0, 0.0, 1)

            with open(output_file, "wb") as f:
                f.write(audio_data)

            end_time = time.time()
            elapsed_time = end_time - start_time

            print(f"\n音声が '{output_file}' に保存されました。")
            print(f"音声合成にかかった時間: {elapsed_time:.2f} 秒")
        except Exception as e:
            print(f"音声合成に失敗しました: {e}")

if __name__ == "__main__":
    main()